[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/7.lm/HW6_Perplexity.ipynb)

# HW6: Perplexity

In this homework, you will implement a function to calculate the perplexity of the n-gram language models we covered in lab, and experiment with different sequences to better understand both n-gram LMs as well as the perplexity metric.

In [1]:
import copy
from collections import Counter

import nltk
import numpy as np
from nltk import sent_tokenize, word_tokenize

nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
def read_file(filename):
    sequences = []
    with open(filename) as file:
        data = file.read()
        sents = sent_tokenize(data)
        for sent in sents:
            tokens = word_tokenize(sent)
            sequences.append(tokens)
    return sequences

In [3]:
!wget --no-check-certificate https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/1342_pride_and_prejudice.txt

--2025-10-03 03:19:21--  https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/1342_pride_and_prejudice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691804 (676K) [text/plain]
Saving to: ‘1342_pride_and_prejudice.txt’

1342_pride_and_prej 100%[===================>] 675.59K  --.-KB/s    in 0.005s  

2025-10-03 03:19:21 (142 MB/s) - ‘1342_pride_and_prejudice.txt’ saved [691804/691804]



In [4]:
# Read data from file and tokenize them into sequences comprised of tokens.

# Pride and Prejudice (Jane Austen)
sequences = read_file("1342_pride_and_prejudice.txt")

max_sequences = 10000

In [5]:
class NgramModel():

    def __init__(self, sequences, order):

        # For this exercise we're going to encode the LM as a sparse dictionary (trading less storage for more compute)
        # We'll store the LM as a dictionary with the conditioning context as keys; each value is a
        # Counter object that keeps track of the number of times we see a word following that context.
        self.counts = {}

        # Markov order (order 1 = conditioning on previous 1 word; order 2 = previous 2 words, etc.)
        self.order = order

        vocab = {"[END]": 0}

        for s_idx, tokens in enumerate(sequences):
            # We'll add [START] and [END] tokens to encode the beginning/end of sentences
            tokens = ["[START]"] * order + tokens + ["[END]"]

            if s_idx == 0:
                print(tokens)

            for i in range(order, len(tokens)):
                context = " ".join(tokens[i - order:i])
                word = tokens[i]

                if word not in vocab:
                    vocab[word] = len(vocab)

                # For just the first sentence, print the conditioning context + word
                if s_idx == 0:
                    print("Context: %s Next: %s" % (context.ljust(50), word))

                if context not in self.counts:
                    self.counts[context] = Counter()
                self.counts[context][word] += 1



    def sample(self, context):
        total = sum(self.counts[context].values())

        dist = []
        vocab = []

        # Create a probability distribution for each conditioning context, over the vocab that we've observed it with.
        for idx, word in enumerate(self.counts[context]):
            prob = self.counts[context][word]/total
            dist.append(prob)
            vocab.append(word)

        index = np.argmax(np.random.multinomial(1, pvals=dist))
        return vocab[index]

    def generate_sequence(self, keep_ends=True):
        generated = ["[START]"] * (self.order)
        word = None
        while word != "[END]":
            context = ' '.join(generated[-self.order:] if self.order > 0 else "")
            word = self.sample(context)
            generated.append(word)
        if not keep_ends:
            generated = generated[self.order:-1]
        return " ".join(generated)



Let's create some language models of different orders from *Pride and Prejudice*.

In [6]:
ngram0 = NgramModel(sequences[:max_sequences], order=0)

['Chapter', '1', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that', 'a', 'single', 'man', 'in', 'possession', 'of', 'a', 'good', 'fortune', ',', 'must', 'be', 'in', 'want', 'of', 'a', 'wife', '.', '[END]']
Context:                                                    Next: Chapter
Context:                                                    Next: 1
Context:                                                    Next: It
Context:                                                    Next: is
Context:                                                    Next: a
Context:                                                    Next: truth
Context:                                                    Next: universally
Context:                                                    Next: acknowledged
Context:                                                    Next: ,
Context:                                                    Next: that
Context:                                                   

In [7]:
ngram1 = NgramModel(sequences[:max_sequences], order=1)

['[START]', 'Chapter', '1', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that', 'a', 'single', 'man', 'in', 'possession', 'of', 'a', 'good', 'fortune', ',', 'must', 'be', 'in', 'want', 'of', 'a', 'wife', '.', '[END]']
Context: [START]                                            Next: Chapter
Context: Chapter                                            Next: 1
Context: 1                                                  Next: It
Context: It                                                 Next: is
Context: is                                                 Next: a
Context: a                                                  Next: truth
Context: truth                                              Next: universally
Context: universally                                        Next: acknowledged
Context: acknowledged                                       Next: ,
Context: ,                                                  Next: that
Context: that                                   

In [8]:
ngram2 = NgramModel(sequences[:max_sequences], order=2)

['[START]', '[START]', 'Chapter', '1', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that', 'a', 'single', 'man', 'in', 'possession', 'of', 'a', 'good', 'fortune', ',', 'must', 'be', 'in', 'want', 'of', 'a', 'wife', '.', '[END]']
Context: [START] [START]                                    Next: Chapter
Context: [START] Chapter                                    Next: 1
Context: Chapter 1                                          Next: It
Context: 1 It                                               Next: is
Context: It is                                              Next: a
Context: is a                                               Next: truth
Context: a truth                                            Next: universally
Context: truth universally                                  Next: acknowledged
Context: universally acknowledged                           Next: ,
Context: acknowledged ,                                     Next: that
Context: , that                      

**Q1.** Create a `perplexity` function that can takes two arguments: a.) a model of *any* ngram order (from the class above); and b.) a sequence to calculate perplexity for.  You'll recall from class that perplexity under a particular language model for sequence $w$ is given by the following equation:

$$
\textrm{perplexity}_{model}(w) = \exp\left(-{1 \over N} \sum_{i=1}^N \log P_{model}(w_i) \right)
$$

$P_{model}(w_i)$ calculates the probability of token $w_i$ using whatever assumptions that model makes -- for a bigram model (order 1), this is $P(w_i \mid w_{i-1})$, for a trigram model (order 2), this is $P(w_i \mid w_{i-2}, w_{i-1})$, etc.  Two things to note:

* When calculating the probability of the first word(s), be sure to get the conditioning context right.  The conditioning context for the first word in a trigram model, for example, is $P(w_i \mid$ [START] [START]$)$.
* Perplexity is only calculated for the words in the actual sequence.  We don't include the $P$([START]) or $P$([END]) in the perplexity calculuation.


*Hint*: when working on this function, you might want to debug by printing out the probabilities of each $w_i$.

In [14]:
from math import log
import numpy as np  # New Line: we'll need numpy for exp at the end

def perplexity(model, tokens):
    #add [START] and [END] tokens like the model expects
    seq = ["[START]"] * model.order + tokens + ["[END]"]
    log_prob = 0.0
    N = len(tokens) + 1
    for i in range(model.order, len(seq)):
        context = " ".join(seq[i - model.order:i])
        word = seq[i]
        total_count = sum(model.counts.get(context, {}).values())
        word_count = model.counts.get(context, {}).get(word, 0) #get how many times this word followed the context

        #Smoothing: if unseen, use a tiny probability instead of 0, cause previously got an error when trying to run Q3
        if total_count == 0 or word_count == 0:
            prob = 1e-6
        else:
            prob = word_count / total_count
        log_prob += log(prob) #add log(prob) to total log probability


    avg_log_prob = - (log_prob / N) #compute average negative log probability
    return np.exp(avg_log_prob)


**Q2**. Execute that perplexity function for the following language models:

In [15]:
perplexity(ngram0, word_tokenize("She was a great friend of Mr. Bingley."))

np.float64(167.8270499710048)

In [16]:
perplexity(ngram1, word_tokenize("She was a great friend of Mr. Bingley."))

np.float64(17.1873731659743)

In [17]:
perplexity(ngram2, word_tokenize("She was a great friend of Mr. Bingley."))

np.float64(8.240572114787614)

**Q3.** What is the perplexity of "She was a really great friend of Mr. Bingley." in the trigram language model trained above?

Explain in 100 words what behavior is expected (and correct) given how an n-gram language model works and the data we are training it on.

In [18]:
perplexity(ngram2, word_tokenize("She was a really great friend of Mr. Bingley."))

np.float64(154.2011240737594)

**Expected Behaviour:** The perplexity of the trigram model is expected to be higher for “She was a really great friend of Mr. Bingley.” than for the earlier sentence without “really”. This happens because n-gram models estimate probabilities based only on counts of observed contexts in the training text.

If a context like “was a really” or “a really great” is rare or unseen in Pride and Prejudice, the model assigns it very low probability. This drives up perplexity. This behavior is correct, since n-gram models cannot generalize beyond patterns they have explicitly observed.

**Q4.** What 1-token sequence yields the lowest perplexity for the 0-order n-gram model? Why?

In [19]:
perplexity(ngram0, word_tokenize("YOUR_ANSWER_HERE"))

np.float64(5613.563566126232)

The 1-token sequence with the lowest perplexity under the 0-order model is simply the most frequent token in the training corpus (often a comma "," with NLTK tokenization, otherwise a very common word like "the").

**Q5.** Write a function to find the n-token sequence with the lowest perplexity given an n-gram model. Explain why it should work.

In [25]:
def min_ppl_sequence(model, n=5):
    """Return an n-token long string that yields the lowest possible perplexity under the provided model."""

    all_tokens = [tok for context in model.counts for tok in model.counts[context]] #create a list of all tokens from training data

    best_seq = None
    best_pp = float("inf")  #start with a very large value

    for i in range(len(all_tokens) - n + 1): #go through every possible n-length slice of tokens
        candidate = all_tokens[i:i+n]
        pp = perplexity(model, candidate)

        if pp < best_pp: #if this sequence has lower perplexity, remember it
            best_pp = pp
            best_seq = candidate

    return best_seq

This function works because - perplexity is lowest when the model assigns the highest probability to a sequence. The n-gram model estimates probabilities from observed counts in the training data.

By sliding over all possible n-token sequences and calculating perplexity, the function identifies the one that is most predictable according to the model. This is expected behavior, since the model is built to favor sequences it has seen often in the corpus.

As a result, the lowest perplexity sequence will usually be a very common phrase, which the model can predict with high confidence.

In [26]:
min_ppl_sequence(ngram1, n=3)

['Mr.', 'Wickham', '.']

In [27]:
perplexity(ngram1, min_ppl_sequence(ngram1, n=3))

np.float64(8.625559780870997)